# Пример верификации человека на фотографии с помощью библиотеки dlib

Верификация - это задача определения, находится ли на изображении нужный нам человек, или нет.

Мы будем решать задачу верификации человека на двух фотографиях. Нам нужно будет определить, один человек изображен на двух фотографиях, или нет.

In [1]:
#!conda install -c conda-forge dlib

In [2]:
import dlib
from skimage import io
from scipy.spatial import distance

## Создаем модели для поиска и нахождения лиц в dlib

Предварительно обученные модели можно скачать по ссылкам:

[Модель для выделения лиц по 68 ключевым точкам](http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2) 
    
[Нейронная сеть ResNet34 для распознавания лиц](http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2) 

Файлы с моделями нужно разархивировать и положить в каталог с notebook

Создаем модели для распознавания лиц на основе ранее загруженных файлов:

In [3]:
sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
detector = dlib.get_frontal_face_detector()

Напишем функцию для загрузки фото, поиска лица и извлечения дескриптора

In [4]:
def getPhoto(img_path):
    img = io.imread(img_path)
    win1 = dlib.image_window()
    win1.clear_overlay()
    win1.set_image(img)
    dets_webcam = detector(img, 1)
    for k, d in enumerate(dets_webcam):
        print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
                k, d.left(), d.top(), d.right(), d.bottom()))
    shape = sp(img, d)
    win1.clear_overlay()
    win1.add_overlay(d)
    win1.add_overlay(shape)
    face_descriptor = facerec.compute_face_descriptor(img, shape)
    return face_descriptor

Загружаем и показываем первую фотографию. Это будет фотография удостоверения личности:

<img src='udv.jpg'>

In [9]:
fd = getPhoto('udv.jpg')

Detection 0: Left: 44 Top: 175 Right: 151 Bottom: 283


Можно напечатать дескриптор и посмотреть, как он выглядит:

In [7]:
print(fd)

-0.0771049
0.065285
0.0309488
-0.0370083
0.00955118
-0.0114499
0.000110027
-0.0847156
0.132765
-0.0838252
0.202945
-0.0144117
-0.284247
-0.0558588
0.0170282
0.10459
-0.079183
-0.159464
-0.0899877
0.0187306
0.125821
-0.0402909
0.0165596
0.0342636
-0.106246
-0.241602
-0.0958455
-0.0799437
-0.0574318
-0.0209923
-0.0502574
0.00248446
-0.154546
-0.0307295
0.0414287
0.0831917
-0.0721541
-0.141036
0.289932
-0.00110112
-0.1522
-0.0496348
0.0524119
0.176063
0.136312
0.0143042
0.00944779
-0.0507398
0.117291
-0.234684
0.0911167
0.132672
0.188599
0.130452
0.0871851
-0.189929
0.0908159
0.154371
-0.227968
0.0248948
-0.0231478
-0.140363
-0.0633948
-0.070928
0.189218
0.147146
-0.110395
-0.140403
0.188762
-0.0823067
-0.0870086
0.0976417
-0.163077
-0.223851
-0.243198
0.0725582
0.350424
0.190694
-0.211127
-0.000116657
0.0182665
-0.0352477
0.0801457
0.0878295
-0.0430876
-0.0115789
-0.099417
-0.00926254
0.176005
-0.0220848
-0.116507
0.163242
0.0131932
0.049626
0.0324228
-0.0107915
0.0146748
0.0112529
-0.17

Загружаем и обрабатываем вторую фотографию

<img src='1.jpg'>

In [10]:
fd2 = getPhoto('1.jpg')

Detection 0: Left: 83 Top: 113 Right: 135 Bottom: 164


Рассчитываем Евклидово расстояние между двумя дексрипторами лиц

В dlib рекомендуется использовать граничное значение Евклидова расстояния между дескрипторами лиц равное 0.6. 
Если Евклидово расстояние меньше 0.6, значит фотографии принадлежат одному человеку.

In [11]:
a = distance.euclidean(fd, fd2)
print(a)

0.353451244939058


Одназначно это один и тот же человек

Попробуем разные фотографии

<img src='1.png'>

In [13]:
fd3 = getPhoto('1.png')

Detection 0: Left: 78 Top: 38 Right: 129 Bottom: 90


In [14]:
a = distance.euclidean(fd, fd3)
print(a)

0.35310597958194556


<img src='2.jpg'>

In [15]:
fd4 = getPhoto('2.jpg')
a = distance.euclidean(fd, fd4)
print(a)

Detection 0: Left: 426 Top: 546 Right: 533 Bottom: 653
0.48663924860117075


<img src='3.jpg'>

In [16]:
fd5 = getPhoto('3.jpg')
a = distance.euclidean(fd, fd5)
print(a)

Detection 0: Left: 106 Top: 116 Right: 195 Bottom: 206
0.4035034187142069


И снова везде я. Поробуем не мои фото.

<img src='k.jpg'>

In [18]:
k = getPhoto('k.jpg')
a = distance.euclidean(fd, k)
print(a)

Detection 0: Left: 27 Top: 170 Right: 348 Bottom: 491
0.8272546743740238


<img src='o.jpg'>

In [19]:
o = getPhoto('o.jpg')
a = distance.euclidean(fd, o)
print(a)

Detection 0: Left: 12 Top: 81 Right: 167 Bottom: 236
0.9863801458574821


На этих фото явно не я

Проверка на очень похожих людях и сильно изменившихся по сравнению с фото на паспорте.

<img src='passport1.jpg'>

In [22]:
fd = getPhoto('passport1.jpg')

Detection 0: Left: 221 Top: 222 Right: 407 Bottom: 407


<img src="passport2.jpg">

In [23]:
fd2 = getPhoto('passport2.jpg')

Detection 0: Left: 216 Top: 167 Right: 439 Bottom: 390


In [24]:
a = distance.euclidean(fd, fd2)
print(a)

0.490645105638294


Один и тот же человек не смотря на то, что внешне сильно не похож.

<img src ="d1.jpg">

In [25]:
fd = getPhoto('d1.jpg')

Detection 0: Left: 81 Top: 98 Right: 236 Bottom: 253


<img src ="d2.jpg">

In [26]:
fd2 = getPhoto('d2.jpg')

Detection 0: Left: 35 Top: 77 Right: 221 Bottom: 263


In [27]:
a = distance.euclidean(fd, fd2)
print(a)

0.5012867697949589


Здесь вышел прокол. Хоть и разные люди, но засчитало как похожего.

Создадим телеграм бота бля верификации фото

In [28]:
import telebot

bot = telebot.TeleBot("")

In [29]:
def getPhotoBot(img_path):
    img = io.imread("https://api.telegram.org/file/bot/"+img_path)
    dets_webcam = detector(img, 1)
    for k, d in enumerate(dets_webcam):
        print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
                k, d.left(), d.top(), d.right(), d.bottom()))
    shape = sp(img, d)
    face_descriptor = facerec.compute_face_descriptor(img, shape)
    return face_descriptor

In [30]:
i = 1
fd1 = ""
fd2 = ""

In [ ]:
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    bot.reply_to(message, "Howdy, how are you doing?")

@bot.message_handler(func=lambda message: True)
def echo_all(message):
    bot.reply_to(message, message.text)

@bot.message_handler(content_types=['photo'])
def handle_docs_photo(message):
    global i, fd1, fd2
    try:
        file_info = bot.get_file(message.photo[len(message.photo)-1].file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        if i % 2 != 0: 
            fd1 = first_photo(file_info.file_path)
        else:
            fd2 = second_photo(file_info.file_path)
        if fd1 != "" and fd2 != "":
            a = distance.euclidean(fd1, fd2)
            bot.reply_to(message,a) 
        i = i + 1
        print(i)
        print(type(fd1))
        print(type(fd2))
    except Exception as e:
        bot.reply_to(message,e )    
    
bot.polling()